In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.graphics.regressionplots import abline_plot
import statsmodels.stats.outliers_influence as st_inf
from sklearn.linear_model import LinearRegression
%matplotlib inline

# 3.6.2 Simple Linear Regression

Load the Boston housing dataset and perform a basic regression. The book uses R. I'm going to use statsmodels and scikit-learn

In [ ]:
df = sm.datasets.get_rdataset("Boston", "MASS", cache=True).data
df = sm.add_constant(df)

In [ ]:
df.head()

## scikit-learn implementation

Not happy that I had to do this weird reshape just because I only had one independent variable. Maybe there's a better way?

In [ ]:
%%timeit
boston_skl_ols = LinearRegression()
boston_skl_ols.fit(df["lstat"].to_numpy().reshape(-1, 1), df["medv"])

In [ ]:
boston_skl_ols = LinearRegression()
boston_skl_ols.fit(df["lstat"].to_numpy().reshape(-1, 1), df["medv"])
print("Coefficients: \n", boston_skl_ols.coef_)
print("Intercept: \n", boston_skl_ols.intercept_)

While it's not as verbose in estimation output, or at least I don't see an easy way to make it be, it's sure a lot faster, which makes sense given what it's designed for.

I think I'll stick with StatsModels for the rest of this

## StatsModels Implementation

Where possible I prefer to use StatsModels. Scikit is great, and if I wanted to do a pure prediction I might prefer it, but StatsModels gives me all that analytic goodness I'm looking for.

In [ ]:
%%timeit
boston_sm_ols = sm.OLS(df["medv"], df[["const", "lstat"]]).fit()

### Run regression and display basic summary stats

In [ ]:
boston_sm_ols = sm.OLS(df["medv"], df[["const", "lstat"]]).fit()
print(boston_sm_ols.summary())

A lot of the details are available in summary, but they can also be accessed using methods if you want to use them in further programming

In [ ]:
boston_sm_ols.conf_int(alpha=0.05)

Calculate confidence and prediction intervals for a set of independent variables (note the call to add_constant to include the intercept)

In [ ]:
boston_sm_ols.get_prediction(sm.add_constant([5, 10, 15])).summary_frame(alpha=0.05)

To plot the regression there is a way to do it in StatsModels, but the nicer way is probably Seaborn, let's try both

In [ ]:
sns.regplot(x='lstat', y='medv', data=df);

In [ ]:
ax = df.plot(x='lstat', y='medv', kind='scatter')
abline_plot(model_results=boston_sm_ols, ax=ax);

Analyse residuals

In [ ]:
# make a new dataframe for easier plotting
result_df = df[["lstat", "medv"]].copy()
result_df["fitted"] = boston_sm_ols.fittedvalues
result_df["resid"] = boston_sm_ols.resid

Basic residuals

In [ ]:
result_df.plot(x="lstat", y="resid", kind="scatter");

Plot leverage against studentized residuals

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
fig = sm.graphics.influence_plot(boston_sm_ols, ax=ax, criterion="cooks")

Can also get leverage and a bunch of other summary stats in numeric form, in ISL the lab shows which observation has the highest leverage

In [ ]:
boston_influence = st_inf.OLSInfluence(boston_sm_ols).summary_frame()

In [ ]:
boston_influence["hat_diag"].idxmax()

ISL returns 375 for this, but I'm guessing that's just because R counts from 1 and python indices are from 0

# 3.6.3 Multiple Linear Regression

Still using StatsModels, going to start using the same variable names as ISL going forward

In [ ]:
lm = sm.OLS(df["medv"], df[["const", "lstat", "age"]]).fit()

In [ ]:
print(lm.summary())

In [ ]:
X = df.drop(columns="medv")
y = df["medv"]
lm = sm.OLS(y, X).fit()
print(lm.summary())

More examples of accessing individual attributes. ISL pulls $R^2$ and RSE so I'll do the same

In [ ]:
print(f"R squared: {lm.rsquared:0.3}, RSE: {lm.mse_resid:0.3}")

Calculate variance inflation factors for all the regressors:

In [ ]:
for i, col in enumerate(X.drop(columns="const").columns):
    var_inf = st_inf.variance_inflation_factor(X.to_numpy(), i + 1) # add one since we dropped constant
    print(f"VIF of {col}: {var_inf:0.2}")

Interaction terms and non linear transformations

I know statsmodels can use R style formulas to define these transformations, but I prefer to do it manually, maybe I'm just oldschool

In [ ]:
X["lstat_x_age"] = X["lstat"] * X["age"]
X["lstat_squared"] = X["lstat"]**2

In [ ]:
print(sm.OLS(y, X[["const", "lstat", "age", "lstat_x_age"]]).fit().summary())

In [ ]:
print(sm.OLS(y, X[["const", "lstat", "lstat_squared"]]).fit().summary())

ANOVA to compare linear and quadratric models

In [ ]:
linear_ols = sm.OLS(y, X[["const", "lstat"]]).fit()
quad_ols = sm.OLS(y, X[["const", "lstat", "lstat_squared"]]).fit()
sm.stats.anova_lm(linear_ols, quad_ols)

In [ ]:
result_df = df[["lstat", "medv"]].copy()
result_df["quad_fitted"] = quad_ols.fittedvalues
result_df["quad_resid"] = quad_ols.resid
result_df["lin_fitted"] = linear_ols.fittedvalues
result_df["lin_resid"] = linear_ols.resid
cdf = result_df[["lstat", "quad_resid", "lin_resid"]].melt(id_vars="lstat", var_name="model", value_name="resid")
sns.scatterplot(x="lstat", y="resid", hue="model", data=cdf);

Sure we've improved the fit, but I don't see it as such a drastic improvement like the text claims

Higher order polynomials

In [ ]:
X_poly = X[["const", "lstat"]].copy()
for i in range(2, 6):
    new_name = "lstat_pow_" + str(i)
    X_poly[new_name] = X_poly["lstat"]**i
poly_ols = sm.OLS(y, X_poly).fit()
print(poly_ols.summary())

Ok, that's way different than the book. The actual model fit is fine, but the coefficients are different. Further reading indicates that poly in R does an orthogonal polynomial. Quick googling didn't turn up any baked in way to compute that in python, but I found [this post](http://davmre.github.io/blog/python/2013/12/15/orthogonal_poly) that gives a method. Let's try that.

In [ ]:
def ortho_poly_fit(x, degree = 1):
    n = degree + 1
    x = np.asarray(x).flatten()
    if(degree >= len(np.unique(x))):
            stop("'degree' must be less than number of unique points")
    xbar = np.mean(x)
    x = x - xbar
    X = np.fliplr(np.vander(x, n))
    q,r = np.linalg.qr(X)

    z = np.diag(np.diag(r))
    raw = np.dot(q, z)

    norm2 = np.sum(raw**2, axis=0)
    alpha = (np.sum((raw**2)*np.reshape(x,(-1,1)), axis=0)/norm2 + xbar)[:degree]
    Z = raw / np.sqrt(norm2)
    return Z, norm2, alpha

def ortho_poly_predict(x, alpha, norm2, degree = 1):
    x = np.asarray(x).flatten()
    n = degree + 1
    Z = np.empty((len(x), n))
    Z[:,0] = 1
    if degree > 0:
        Z[:, 1] = x - alpha[0]
    if degree > 1:
      for i in np.arange(1,degree):
          Z[:, i+1] = (x - alpha[i]) * Z[:, i] - (norm2[i] / norm2[i-1]) * Z[:, i-1]
    Z /= np.sqrt(norm2)
    return Z

In [ ]:
lstat = X["lstat"]
X_poly, _, _ = ortho_poly_fit(lstat, degree=5)
poly_ols = sm.OLS(y, sm.add_constant(X_poly[:, 1:])).fit()
print(poly_ols.summary())

Ok, that fits now. The first column in the returned array is a weirdly scaled intercept, which is why I had to drop that column and add a constant back in... 
Not super elegant, but it does get the job done